In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler
import copy

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### TOPIC MODELING w/ LDA and Dynamic LDA

#### a) preprocessing

In [2]:
# LDA topic modeling

import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldaseqmodel import LdaSeqModel
import spacy
import pickle
import math
import ray
ray.init(num_cpus=12)

pyLDAvis.enable_notebook()
nlp = spacy.load('en_core_web_sm')

In [3]:
doc_df = pd.read_csv('tables/tables_69_76/doc.csv')
doc_df = doc_df[doc_df['subtype']!='editorial-note']
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

In [4]:
@ray.remote
def preprocess(row):
    return [t.lemma_.lower() for t in nlp(row) if not t.is_punct and not t.is_stop and t.ent_iob_=='O' and len(t.text)>=3]
# and t.pos_ in ['PROPN','NOUN']

futures = [preprocess.remote(row) for row in free_text_list]
processed_free_text_list = ray.get(futures)

2023-03-28 14:10:38,718	WARNING worker.py:1257 -- Warning: The remote function __main__.preprocess is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


In [ ]:
'''with open("tables/tables_69_76/lda_processed_entremoved_onlynoun_free_text_list", "wb") as fp:
    pickle.dump(processed_free_text_list, fp)'''

In [5]:
# for dynamic lda

year_list = list(map(lambda x: int(x),doc_df['year']))
year_text_tuple = list(zip(year_list, processed_free_text_list))
year_text_tuple = sorted(year_text_tuple, key=lambda x: x[0])
sorted_year_list = list(map(lambda x: x[0],year_text_tuple))
sorted_text_list = list(map(lambda x: x[1],year_text_tuple))

#### b.1) gensim

In [6]:
# LDA
dictionary = Dictionary(processed_free_text_list)

dictionary.filter_extremes(keep_n=10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_free_text_list]

lda = LdaModel(corpus = doc_term_matrix, id2word=dictionary, num_topics=30)

gensimvis.prepare(lda, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12     0.127030  0.037565       1        1  9.274050
29    -0.025619  0.077944       2        1  6.274509
17    -0.201994 -0.042956       3        1  6.073815
18    -0.037899  0.045297       4        1  5.271811
22    -0.129346  0.033152       5        1  4.765971
21    -0.101367 -0.011021       6        1  4.337022
15     0.122870 -0.049550       7        1  4.209142
27    -0.011762 -0.001054       8        1  4.056872
10     0.024470  0.013289       9        1  4.047227
4      0.167446  0.010249      10        1  3.996227
5     -0.152043 -0.008399      11        1  3.942832
13     0.031347  0.041321      12        1  3.767231
9      0.138189  0.081849      13        1  3.722769
8      0.115431  0.045924      14        1  3.496913
2      0.030488  0.048744      15        1  3.048903
1      0.146993 -0.145274      16        1  2.844345
26    -0.074981 -0.107431      17        1  2.833736
25     0.022543 -0.106522      18        1  2.830132
11    -0.028444 -0.100770      19        1  2.748677
3     -0.105976 -0.052107      20        1  2.504680
23    -0.087913  0.036886      21        1  2.082349
20    -0.078159  0.003613      22        1  2.031910
6      0.037139  0.093519      23        1  2.024806
24    -0.009145  0.014961      24        1  1.979278
28     0.117125 -0.084110      25        1  1.956588
14     0.060265 -0.169564      26        1  1.693372
0     -0.023929  0.081564      27        1  1.386733
19     0.076435  0.151466      28        1  1.344475
16    -0.091569  0.050541      29        1  1.186548
7     -0.057623  0.010873      30        1  0.267079, topic_info=           Term          Freq         Total Category  logprob  loglift
373         mr.  47076.000000  47076.000000  Default  30.0000  30.0000
442   secretary  51223.000000  51223.000000  Default  29.0000  29.0000
140         say  46002.000000  46002.000000  Default  28.0000  28.0000
1378        oil  14555.000000  14555.000000  Default  27.0000  27.0000
367    minister  26844.000000  26844.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
82        issue     63.648122  28791.278939  Topic30  -5.6744  -0.1891
351        line     61.029055  20033.634689  Topic30  -5.7165   0.1316
477        talk     58.610767  25117.952879  Topic30  -5.7569  -0.1350
119   political     58.336989  23286.145252  Topic30  -5.7616  -0.0640
167        tell     56.884552  23338.500220  Topic30  -5.7868  -0.0915

[2330 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
4251      3  0.112526  10:00
4251      5  0.843945  10:00
5009      6  0.791973   1954
5009     10  0.049498   1954
5009     12  0.049498   1954
...     ...       ...    ...
1071     25  0.000346   zone
1071     26  0.098180   zone
1071     27  0.000691   zone
1071     29  0.000346   zone
1071     30  0.000346   zone

[21694 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 30, 18, 19, 23, 22, 16, 28, 11, 5, 6, 14, 10, 9, 3, 2, 27, 26, 12, 4, 24, 21, 7, 25, 29, 15, 1, 20, 17, 8])

In [ ]:
# dynamic LDA (Gensim's dynamic lda is too slow. this code below will not terminate)
dictionary = Dictionary(sorted_text_list)

dictionary.filter_extremes(keep_n=100)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in sorted_text_list]

time_slice = pd.Series(sorted_year_list).value_counts(sort=False).values

ldaseq = LdaSeqModel(corpus = doc_term_matrix, time_slice=list(time_slice), id2word=dictionary, num_topics=5)

#### b.2) tomotopy

In [38]:
# LDA
import tomotopy as tp
import sys

# define model
mdl = tp.LDAModel(k=30,tw=tp.TermWeight.ONE)

# add documents
for txt in processed_free_text_list:
    mdl.add_doc(txt)

# train model
mdl.train(100)

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency,start_index=0,sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0     -0.002147  0.220883       0        1  5.737029
1      0.178072  0.013821       1        1  0.841704
2      0.055222  0.069889       2        1  2.677456
3     -0.106711  0.054776       3        1  5.949531
4      0.109813 -0.053170       4        1  2.628150
5      0.124422 -0.168752       5        1  2.667158
6      0.115863 -0.009952       6        1  2.543681
7     -0.143857  0.058449       7        1  1.440431
8      0.177098 -0.109569       8        1  3.664908
9     -0.055730  0.084952       9        1  5.339519
10     0.014939 -0.030818      10        1  6.169053
11    -0.199840 -0.136929      11        1  2.784269
12     0.161553 -0.044806      12        1  6.001372
13    -0.289356 -0.131614      13        1  2.086178
14     0.088422  0.158571      14        1  4.187090
15    -0.002235  0.067162      15        1  5.882910
16    -0.282356 -0.078800      16        1  2.793855
17    -0.252855  0.035328      17        1  3.756806
18     0.174186 -0.016434      18        1  1.230322
19    -0.302145 -0.056782      19        1  4.651464
20     0.108072  0.145004      20        1  1.521306
21    -0.209785  0.076669      21        1  2.387692
22     0.025333 -0.134142      22        1  3.328017
23     0.151735  0.097414      23        1  2.940023
24    -0.158873  0.143490      24        1  3.033175
25     0.079987 -0.092983      25        1  3.214792
26     0.143503 -0.104020      26        1  2.207663
27     0.096145  0.071661      27        1  1.064949
28     0.120739 -0.023741      28        1  2.390111
29     0.080786 -0.105557      29        1  4.879383, topic_info=         Term          Freq         Total Category  logprob  loglift
1         mr.  52585.000000  52585.000000  Default  30.0000  30.0000
2   secretary  50410.000000  50410.000000  Default  29.0000  29.0000
3         say  49060.000000  49060.000000  Default  28.0000  28.0000
0   president  58498.000000  58498.000000  Default  27.0000  27.0000
14        dr.  27799.000000  27799.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
30   continue   1410.003993  21334.698255  Topic29  -5.5236   0.3034
51    present   1234.378502  16759.033891  Topic29  -5.6566   0.4118
72     change   1168.518892  14020.878909  Topic29  -5.7115   0.5353
0   president   1282.276386  58498.923734  Topic29  -5.6186  -0.8002
36    general   1062.744378  20337.602490  Topic29  -5.8063   0.0685

[2525 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
9187       8  0.991050        1/3
10234      8  0.987951        110
11579     14  0.985409        112
5548      27  0.992136      11652
11005      8  0.985640  11–3/8–75
...      ...       ...        ...
722       25  0.141508       zone
16187      7  0.963027    zumwalt
6981       2  0.996219     émigré
7688       2  0.994794    émigrés
6372      13  1.003306        ’em

[9660 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [40]:
doc_inst = mdl.docs[0]

In [46]:
mdl.get_topic_words(17)

[('say', 0.1309700310230255),
 ('president', 0.04592324048280716),
 ('ask', 0.03636263683438301),
 ('note', 0.016635632142424583),
 ('point', 0.015993118286132812),
 ('problem', 0.015185386873781681),
 ('reply', 0.015020170249044895),
 ('want', 0.012909053824841976),
 ('tell', 0.012497845105826855),
 ('think', 0.011403736658394337)]

In [43]:
doc_inst.get_topics()

[(13, 0.16549718379974365),
 (0, 0.15226240456104279),
 (19, 0.15165044367313385),
 (10, 0.1446617841720581),
 (24, 0.12272237241268158),
 (15, 0.09451723098754883),
 (11, 0.06528868526220322),
 (8, 0.029181264340877533),
 (12, 0.02236236073076725),
 (17, 0.022185254842042923)]

In [14]:
# dynamic LDA (very fast)

# define model
mdl = tp.DTModel(k=30, t=len(np.unique(sorted_year_list)), tw=tp.TermWeight.ONE)

# add documents
for idx,txt in enumerate(sorted_text_list):
    time_pt = sorted_year_list[idx]-min(sorted_year_list)
    mdl.add_doc(words=txt, timepoint=time_pt)

# train model
mdl.train(100)

2023-03-28 16:54:27,680	WARNING worker.py:469 -- The driver may not be able to keep up with the stdout/stderr of the workers. To avoid forwarding logs to the driver, use 'ray.init(log_to_driver=False)'.


In [36]:
for t in range(mdl.num_timepoints):
    results = mdl.get_topic_words(topic_id=15,timepoint=t,top_n=5)
    print(f't:{t}, top words: {list(map(lambda x: x[0],results))}')

t:0, top words: ['time', 'standard', 'ask', 'give', 'feel']
t:1, top words: ['give', 'ask', 'feel', 'conversation', 'chance']
t:2, top words: ['ask', 'give', 'feel', 'wish', 'say']
t:3, top words: ['say', 'ask', 'government', 'feel', 'war']
t:4, top words: ['ask', 'decision', 'feel', 'result', 'accept']
t:5, top words: ['agreement', 'agree', 'position', 'ask', 'accept']
t:6, top words: ['ask', 'agree', 'position', 'agreement', 'secretary']
t:7, top words: ['secretary', 'ask', 'agree', 'position', 'agreement']
t:8, top words: ['secretary', 'state', 'ask', 'negotiation', 'agree']
t:9, top words: ['secretary', 'state', 'know', 'right', 'intervene']
t:10, top words: ['state', 'secretary', 'talk', 'nature', 'salt']
t:11, top words: ['state', 'oil', 'instruction', 'multiple', 'provide']
t:12, top words: ['oil', 'state', 'subject', 'statement', 'company']
t:13, top words: ['oil', 'market', 'supply', 'world', 'state']
t:14, top words: ['oil', 'concern', 'price', 'time', 'believe']


### below is for parallel removal of named entities from free texts. (save in original code)

In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
import math
import ray
ray.init(num_cpus=12)

nlp = spacy.load('en_core_web_sm')

name_extension = '_52_88_entremoved'

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
@ray.remote
def remove_entities(txt):
    document = nlp(txt)

    edited_txt = ""
    for token in document:
        
        if token.ent_iob_=='O':
            if token.whitespace_:
                edited_txt += token.text+ ' '
            else:
                edited_txt += token.text
    
    return edited_txt

In [3]:
doc_df = pd.read_csv('tables/tables_52_88/doc.csv')
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

futures = [remove_entities.remote(txt) for txt in free_text_list]
free_text_list = ray.get(futures)
print('entities removed from free texts.')

with open("tables/tables_52_88/free_text_list"+name_extension, "wb") as fp:
    pickle.dump(free_text_list, fp)

2023-03-27 13:39:04,684	WARNING worker.py:1257 -- Warning: The remote function __main__.remove_entities is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


entities removed from free texts.


### INSTITUTION EXTRACTION